# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution

In [1]:
def square_root_generator(limit):
    n = 1
    sum = 0
    while n <= limit:
        sum += n ** 0.5 
        yield sum
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762


In [3]:
def square_root_generator(limit):
    n = 1
    sum = 0
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


In [2]:
import pandas as pd
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data_1 = pd.DataFrame(people_1())
data_2 = pd.DataFrame(people_2())

data = pd.concat([data_1,data_2])

con = duckdb.connect(database=':memory:', read_only=False)

con.register('people', data)

query = "SELECT SUM(Age) FROM people"
result = con.execute(query).fetchall()

sum_of_ages = result[0][0]

print(sum_of_ages)

353


In [4]:
con = duckdb.connect(database=':memory:', read_only=False)

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data_1 = pd.DataFrame(people_1())

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data_2 = pd.DataFrame(people_2())

con.register('people', data_1)
con.register('people', data_2, overwrite=True)


query = "SELECT SUM(Age) FROM people"
result = con.execute(query).fetchall()

sum_of_ages = result[0][0]

print(sum_of_ages) 

TypeError: register(): incompatible function arguments. The following argument types are supported:
    1. (self: duckdb.duckdb.DuckDBPyConnection, view_name: str, python_object: object) -> duckdb.duckdb.DuckDBPyConnection

Invoked with: <duckdb.duckdb.DuckDBPyConnection object at 0x0000022F5E01A170>, 'people',    ID      Name  Age    City Occupation
0   3  Person_3   33  City_B      Job_3
1   4  Person_4   34  City_B      Job_4
2   5  Person_5   35  City_B      Job_5
3   6  Person_6   36  City_B      Job_6
4   7  Person_7   37  City_B      Job_7
5   8  Person_8   38  City_B      Job_8; kwargs: overwrite=True

In [8]:
con = duckdb.connect(database=':memory:', read_only=False)

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

data_1 = pd.DataFrame(people_1())

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

data_2 = pd.DataFrame(people_2())

merged_data = pd.concat([data_1, data_2]).drop_duplicates(subset='ID', keep='last')

con.register('people', merged_data)

query = "SELECT SUM(Age) FROM people"
result = con.execute(query).fetchall()

sum_of_ages = result[0][0]

print(sum_of_ages)

266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX